In [8]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import psycopg2

In [9]:
season = 2023

In [10]:
# Directory named data with 2 subdirectories: standings and scores
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")

In [11]:
async def get_html(url, selector, sleep = 5, retries = 1):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch() # Can also use p.chromium.launch() if issues persist
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [12]:
async def scrape_betting_lines(season):
    url = f"https://www.sportsoddshistory.com/nfl-game-season/?y={season}"
    html = await get_html(url, "#content .entry_content")
    
    save_path = os.path.join(STANDINGS_DIR, f"{season}-games-lines.htm") # Designate the filename and path
            
    with open(save_path, "w+") as f:
        f.write(html)
        
    return save_path

In [13]:
filepath = await scrape_betting_lines(season)

Historical NFL Game Odds by Season |SportsOddsHistory.com


In [20]:
with open(filepath, 'r') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

# Find all table elements
tables = soup.find_all('table', class_='soh1')

# Extract data from all tables and store it in a list of dictionaries
data = []
for table in tables:
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        row_data = {
            'Day': columns[0].text,
            'Date': columns[1].text,
            'Time (ET)': columns[2].text,
            'Favorite': columns[4].text,
            'Score': columns[5].text,
            'Spread': columns[6].text,
            'Underdog': columns[8].text,
            'Over/Under': columns[9].text,
        }
        data.append(row_data)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)

IndexError: list index out of range

In [19]:
table

<table class="soh1" onmouseout='changeback(event, "white")' onmouseover='changeto(event, "#FFFFCC")'>
<tbody><tr bgcolor="#C9C9C9" id="ignore">
<th rowspan="2"><a href="https://www.sportsoddshistory.com/nfl-game-season/?y2023=&amp;o=w">Week</a></th>
<th colspan="2">Favorites</th>
<th colspan="2">Home<br/>Teams</th>
<th colspan="2">Home<br/>Favorites</th>
<th colspan="2">Home<br/>Underdogs</th>
<th rowspan="2"><a href="https://www.sportsoddshistory.com/nfl-game-season/?y=2023&amp;o=ou">Over/<br/>Unders</a></th>
</tr>
<tr bgcolor="#C9C9C9" id="ignore">
<th><a href="https://www.sportsoddshistory.com/nfl-game-season/?y=2023&amp;o=fs">Straight<br/>Up</a>
</th><th><a href="https://www.sportsoddshistory.com/nfl-game-season/?y=2023&amp;o=fa">ATS</a></th>
<th><a href="https://www.sportsoddshistory.com/nfl-game-season/?y=2023&amp;o=hs">Straight<br/>Up</a></th>
<th><a href="https://www.sportsoddshistory.com/nfl-game-season/?y=2023&amp;o=ha">ATS</a></th>
<th><a href="https://www.sportsoddshistory.